In [1]:
import jinja2
import os
import csv
import pandas as pd
import json
import sys
import argparse
import copy

In [2]:
def render_index(tpl_path, context):
    path, filename = os.path.split(tpl_path)

    environment = jinja2.Environment(undefined=jinja2.StrictUndefined,
    loader=jinja2.FileSystemLoader(path or '.')) #It means jinja will take templates of environment from file in the introduced path
    # Include url_for_.... in the context
    context['url_for_index'] = 'index.html'
    context['url_for_qe'] = 'qe.html'
    context['url_for_yambo'] = 'yambo.html'
    
    return environment.get_template(filename).render(context)

def render_qe(tpl_path, context):
    path, filename = os.path.split(tpl_path)

    environment = jinja2.Environment(undefined=jinja2.StrictUndefined,
    loader=jinja2.FileSystemLoader(path or '.')) #
    # Include url_for_... in the context
    context['url_for_index'] = 'index.html'
    context['url_for_qe'] = 'qe.html'
    context['url_for_yambo'] = 'yambo.html'

    return environment.get_template(filename).render(context)

def render_yambo(tpl_path, context):
    path, filename = os.path.split(tpl_path)

    environment = jinja2.Environment(undefined=jinja2.StrictUndefined,
    loader=jinja2.FileSystemLoader(path or '.')) #
    # Include url_for_index in the context
    context['url_for_index'] = 'index.html'
    context['url_for_qe'] = 'qe.html'
    context['url_for_yambo'] = 'yambo.html'
    
    return environment.get_template(filename).render(context)

In [3]:
def render_template(template_path, entry, entry_info, table_columns, context):
    # Load the template environment
    template_loader = jinja2.FileSystemLoader(searchpath=os.path.dirname(template_path))
    template_env = jinja2.Environment(loader=template_loader, autoescape=True)


    def load_csv_data(entry):
        return entry.to_dict(orient='list')

    template_env.filters['load_csv_data'] = load_csv_data

    def to_list(series, format_numbers=False):
        if format_numbers:
            return [float(f'{num:.2f}') for num in series.tolist()]
        else:
            return series.tolist()
    
    template_env.filters['to_list'] = to_list
    context['table_columns'] = table_columns
    context['Lx_axis'] = entry_info['x_axis']
    context['Lcolumn_name'] = entry_info['column_name']
    context['Lcomponent'] = entry_info['component']
    context['system'] = entry_info['system']
    # Load the template
    template = template_env.get_template(os.path.basename(template_path))

    # Include the dataframe and column_names in the context
    context['dataframe'] = entry
    context['x'] = entry[entry_info['x_axis']]
    context['column_name'] = entry[entry_info['column_name']]
    context['output_path'] = entry_info['output_path']
    if entry_info['component'] != 'empty':
        context['component'] = entry[entry_info['component']]
        context['other_comp'] = entry[entry_info['column_name']] - entry[entry_info['component']]

    for column in table_columns:
        context[f'{column}'] = entry[column]

    # Render the template with the provided context
    output_html = template.render(context)

    # Write the rendered HTML to the output file
    with open(entry_info['output_path'], 'w') as output_file:
        output_file.write(output_html)


def gen_index():
    entries = {}
    with open ('index.html', 'w') as f:
        f.write(render_index('templates/index.tmpl', {'entries': entries}))
        
def gen_qe(flist_qe):
    entries = {}
    #n=0
    for filename in flist_qe:
        _, basename = os.path.split(filename)
        name, _ = os.path.splitext(basename)
        entries[name+'.html'] = name
        #entries[name+f'{n}'+'.html'] = name+f'{n}'
        #n+=1
    with open ('qe.html', 'w') as f:
        f.write(render_qe('templates/qe.tmpl', {'entries': entries}))
        
def gen_yambo(flist_yambo):
    entries = {}
    #n=0
    for filename in flist_yambo:
        _, basename = os.path.split(filename)
        name, _ = os.path.splitext(basename)
        entries[name+'.html'] = name
        #entries[name+f'{n}'+'.html'] = name+f'{n}'
        #n+=1
    with open ('yambo.html', 'w') as f:
        f.write(render_yambo('templates/yambo.tmpl', {'entries': entries}))

def count_files(directory):
    """In order to count the numbers of data_files and get the files in the provided directory"""
    try:
        # List all files in the directory
        files = os.listdir(directory)
        # Count the number of files
        file_count = len(files)
        return file_count, files
    except OSError:
        # Handle the case where the directory doesn't exist or is not accessible
        return -1, []

def code_recogniser(file_path, keywords):
    # Extract the folder names from the file path
    folders = file_path.split(os.path.sep)

    # Find the folder with any of the specified keywords
    found_keywords = [folder for folder in folders if any(keyword.lower() in folder.lower() for keyword in keywords)]

    if found_keywords:
        #print(f"Found keywords {found_keywords} in the folder path.")
        return found_keywords[0]
    else:
        print(f"No specified keywords found in the file path.")
        return None

def struct_recogniser(file_path, position):
    # Extract the folder names from the file path
    folders = file_path.split(os.path.sep)

    # Check if the specified position is valid
    if 0 <= position < len(folders):
        return folders[position]
    else:
        print(f"Invalid position {position}. Please provide a valid position.")
        return None


def dataframes_maker(dataframes_list, **kwargs):
    loaded_dataframes = {}
    for num, df_dict in enumerate(dataframes_list, start=1):
        df_name = f'df_{num}'
        dataframe = pd.read_csv(df_dict['filename'], sep=';')
        code_name= code_recogniser(df_dict['filename'], keywords_to_search)
        system= struct_recogniser(df_dict['filename'], position)
        # Add new information to the dataframe dictionary
        dataframe_info = {
            'dataframe': dataframe,
            'x_axis': df_dict['x_axis'],
            'column_name': df_dict['column_name'],
            'time_unit': df_dict['time_unit'],
            'component': df_dict['component'],
            'code': code_name,
            'system': system,
            'output_path': kwargs.get('output_path', f'./output_chart{num}_{code_name}_{system}.html'),
            #'column_names': {df_dict['column_name']: ' '},  # Adjust as needed
        }
        
        loaded_dataframes[df_name] = copy.deepcopy(dataframe_info)

        # Add more items if needed
       # loaded_dataframes[df_name]['new_item'] = 'some_value'

    return loaded_dataframes

In [8]:
page_title = "Benchmarking results"


dataframes_list = [{'filename': './qe/ausurf/results.dat', 'x_axis': 'Nodes',
                    'column_name': 'walltime', 'time_unit': 'second', 'component': 'sth_kernel', 
                    'code': 'qe', 'system': 'ausurf'}, {'filename': './yambo/Nickel/result1.dat',
                                                        'x_axis': 'Nodes', 'column_name': 'walltime', 
                                                        'time_unit': 'second', 'component': 'electrons',
                                                        'code': 'yambo', 'system': 'Nickel'}, 
                   {'filename': './yambo/Nickel/result0.dat', 'x_axis': 'Nodes', 'column_name': 'walltime',
                    'time_unit': 'second', 'component': 'vloc_psi', 'code': 'yambo', 'system': 'Nickel'}]

keywords_to_search = ['yambo', 'qe', 'othercode']

position = 2

loaded_dataframes = dataframes_maker(dataframes_list)

# Create the context to be used in the template
context = {
        'page_title': page_title,
    }
    # Render the template and save the output HTML
filenames_qe = []
filenames_yambo = []
# Accessing the dataframes and associated information
for df_name, df_info in loaded_dataframes.items():
    dataframe = df_info['dataframe']
    x_axis = df_info['x_axis']
    column_name = df_info['column_name']
    time_unit = df_info['time_unit']
    component = df_info['component']
    code = df_info['code']
    output_path = df_info['output_path']
    table_columns = []
    for i in range(len(dataframe.columns)):
        table_columns.append(dataframe.columns[i])
    #print(f"Processing dataframe: {df_name}")
    #print(f"X-axis: {x_axis}, Column Name: {column_name}, Time Unit: {time_unit}, Component: {component}, Code: {code}")
    #print(dataframe)
    #render_template("templates/chart_modify.tmpl", output_path, context, dataframe=dataframe, column_name = column_name)
    render_template("templates/chart_modify.tmpl", dataframe, df_info, table_columns, context)
    if code=='qe':
        filenames_qe.append(output_path)
    if code=='yambo':
        filenames_yambo.append(output_path)

gen_index()
gen_qe(filenames_qe)
gen_yambo(filenames_yambo)

print(f"Template has been rendered and saved to {filenames_yambo} and {filenames_qe}")


Template has been rendered and saved to ['./output_chart2_yambo_Nickel.html', './output_chart3_yambo_Nickel.html'] and ['./output_chart1_qe_ausurf.html']


In [5]:
data_file_path = ["./df_1.txt", "./df_2.txt"]
    
# Define the output path for the rendered HTML
output_path = ["./output_chart1.html", "./output_chart2.html"]
zipped_list = list(zip(data_file_path, output_path))
zipped_list

[('./df_1.txt', './output_chart1.html'),
 ('./df_2.txt', './output_chart2.html')]

In [6]:
for file_path, output_file in list(zip(data_file_path, output_path)):
    print(f'{file_path}', output_file)

./df_1.txt ./output_chart1.html
./df_2.txt ./output_chart2.html


In [7]:
dataframes_list = [
    {'filename': './qe/cri3-small/result.dat', 'column_name': 'electrons', 'efficiency':'electrons', 'x_axis':'Nodes', 'time_unit':'second'},
    #{'filename': '/result.dat', 'column_name': 'electrons', 'efficiency':'electrons', 'x_axis':'Nodes','time_unit':'second'},
   # {'filename': './qe/ausurf/results.dat', 'column_name': 'sth_kernel', 'efficiency':'sth_kernel', 'x_axis':'Nodes', 'time_unit':'second'}
    ]
loaded_dataframes = {}
for num, df_dict in enumerate(dataframes_list, start=1):
        df_name = f'df_{num}'
        filename = df_dict.get('filename', '')
        if filename:
            loaded_dataframes[df_name] = copy.deepcopy(pd.read_csv(filename, sep=';'))
            print(df_name, df_dict.get('column_name', ''), df_dict.get('x_axis', ''),
                  df_dict.get('efficiency', ''), df_dict.get('time_unit', ''))
print(loaded_dataframes)

df_1 electrons Nodes electrons second
{'df_1':    Nodes  Tasks              install  electrons  cbands  diaghg  Xegterg  \
0      4     16    QE7.2-IOFIX-AWARE     270.07  229.38   18.80   213.30   
1      6     24    QE7.2-IOFIX-AWARE     256.45  215.95   19.10   204.43   
2      8     32    QE7.2-IOFIX-AWARE     197.88  169.06   17.49   160.01   
3      4     16  QE7.2-IOFIX-NOAWARE     213.47  185.17   25.06   169.76   
4      6     24  QE7.2-IOFIX-NOAWARE     185.93  163.53   26.38   152.06   
5      8     32  QE7.2-IOFIX-NOAWARE     163.63  144.14   27.48   134.26   

   vloc_psi   h_psi  sum_band  walltime  vloc_calls  
0    170.74  195.15     40.22    341.78        74.0  
1    168.25  189.01     40.15    316.10        74.0  
2    129.65  145.98     28.54    245.17        74.0  
3    111.30  136.19     27.84    275.15        74.0  
4     98.00  119.96     22.05    236.30        74.0  
5     81.33   99.70     19.22    204.30        74.0  }
